In [1]:
import numpy as np
import cv2
import os
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
from keras import backend as k
#from PIL import Image
#from io import BytesIO
#import base64

In [5]:
base_path = Path("/opt/carla-output")
base_data_path = Path(f'{base_path}/dataset')
base_image_path = Path(f'{base_path}/images')
base_model_path = Path(f'{base_path}/models')

In [4]:
json_dirs = [x for x in base_data_path.iterdir()]
print(json_dirs)

[PosixPath('/opt/carla-output/dataset/12092023-1456'), PosixPath('/opt/carla-output/dataset/08092023-0828'), PosixPath('/opt/carla-output/dataset/12092023-1544'), PosixPath('/opt/carla-output/dataset/06092023-0447'), PosixPath('/opt/carla-output/dataset/11092023-1909'), PosixPath('/opt/carla-output/dataset/12092023-1551'), PosixPath('/opt/carla-output/dataset/01092023-1933'), PosixPath('/opt/carla-output/dataset/06092023-0457'), PosixPath('/opt/carla-output/dataset/08092023-0832'), PosixPath('/opt/carla-output/dataset/08092023-0831'), PosixPath('/opt/carla-output/dataset/user-demand'), PosixPath('/opt/carla-output/dataset/01092023-1930'), PosixPath('/opt/carla-output/dataset/08092023-0829'), PosixPath('/opt/carla-output/dataset/06092023-0452'), PosixPath('/opt/carla-output/dataset/12092023-1502'), PosixPath('/opt/carla-output/dataset/08092023-0819'), PosixPath('/opt/carla-output/dataset/11092023-1910')]


In [ ]:
"""
def substring_after(s, delim):
    return s.partition(delim)[2]
"""

In [6]:
"""
for a, b,c in os.walk(base_model_path):
    split1= substring_after(str(c),"_")
    split2 = substring_after(split1)
"""
now = datetime.now()
date_string = now.strftime("%Y%m%d-%H%M")
print(date_string)

20230912-1701


In [7]:
src_dir = Path('12092023-1543') # Enter the directory name on which to run ML training job
json_path = Path(f'{base_data_path}/{src_dir}')
print(json_path)
json_file_names = []
#json_file = ""
for root, dir_name,json_file in os.walk(json_path):
    json_file_names = json_file

#print(json_file_names)
#print(src_dir)

/opt/carla-output/dataset/12092023-1543


In [8]:
x=[]
y=[]
#dir_name = '01092023-1933'
print(src_dir)
for json_file in json_file_names:
    filename = Path(f'{json_path}/{json_file}')
    #print(filename)
    with open(filename) as f:
        payload = json.load(f)
        #print(payload)
        image_name = payload["rgb_image"]
        #print(image_name)
        image_full_path = f'{base_image_path}/{src_dir}/{image_name}'
        #print(image_full_path)
        x.append(cv2.imread(image_full_path))
        #image_decoded = Image.open(BytesIO(base64.b64decode(image_full_path)))
        #x.append(image_decoded)
        y.append(payload["angle"])


12092023-1543


In [9]:
#Convert into numpy array
x_arr = np.asarray(x)
x_fl = x_arr.astype("float32")
x_norm = x_fl/ 255.0

y_arr = np.asarray(y)

In [ ]:
#print(x_norm.shape)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x_norm, y_arr, test_size=0.1)

In [11]:
print(f'x_train : {x_train.shape}')
print(f'y_train : {y_train.shape}')
print(f'x_test : {x_test.shape}')
print(f'y_test : {y_test.shape}')

x_train : (3100, 480, 640, 3)
y_train : (3100,)
x_test : (345, 480, 640, 3)
y_test : (345,)


In [12]:
########################################################################
tf.__version__

k.clear_session()
del cnn_model
#tf.reset_default_graph()

NameError: name 'cnn_model' is not defined

In [13]:
#Creating the model

cnn_model = tf.keras.models.Sequential()

In [14]:
#Cropping image
cnn_model.add(tf.keras.layers.Cropping2D(cropping=((150,150),(150,150))))


In [15]:
# Adding Convolution Layer
cnn_model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu", padding="SAME",input_shape=[480,640,3]))

#Addning MaxPool Layer
cnn_model.add(tf.keras.layers.MaxPool2D(pool_size=2))

#DropOut
#cnn_model.add(tf.keras.layers.Dropout(0.2))


In [16]:
# Adding Convolution Layer
cnn_model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="SAME"))
cnn_model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="SAME"))

#Addning MaxPool Layer
cnn_model.add(tf.keras.layers.MaxPool2D(pool_size=2))

#DropOut
#cnn_model.add(tf.keras.layers.Dropout(0.2))

In [17]:
#Adding Other Conv and Pooling layers

cnn_model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding="SAME"))
cnn_model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding="SAME"))
#cnn_model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="VALID"))

# MaxPool
cnn_model.add(tf.keras.layers.MaxPool2D(pool_size=2))


In [18]:
#Adding Other Conv and Pooling layers

#cnn_model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding="VALID"))
#cnn_model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding="VALID"))

#cnn_model.add(tf.keras.layers.MaxPool2D(pool_size=2))

In [19]:
#Adding Flattening

cnn_model.add(tf.keras.layers.Flatten())

In [20]:
#Adding Full Conection and dropouts

cnn_model.add(tf.keras.layers.Dense(units=128, activation="relu"))
cnn_model.add(tf.keras.layers.Dropout(0.2))

cnn_model.add(tf.keras.layers.Dense(units=64, activation="relu"))
cnn_model.add(tf.keras.layers.Dropout(0.2))



In [21]:
#Output Layer
cnn_model.add(tf.keras.layers.Dense(units=1))

In [22]:
##################################################################
# Model training

cnn_model.compile(optimizer="SGD", loss="mean_squared_error")



In [23]:
cnn_model.fit(x_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
97/97 [==============================] - 124s 498ms/step - loss: 0.0096
Epoch 2/50
97/97 [==============================] - 36s 376ms/step - loss: 0.0088
Epoch 3/50
97/97 [==============================] - 35s 363ms/step - loss: 0.0085
Epoch 4/50
97/97 [==============================] - 35s 364ms/step - loss: 0.0081
Epoch 5/50
97/97 [==============================] - 35s 357ms/step - loss: 0.0076
Epoch 6/50
97/97 [==============================] - 35s 360ms/step - loss: 0.0073
Epoch 7/50
97/97 [==============================] - 35s 356ms/step - loss: 0.0070
Epoch 8/50
97/97 [==============================] - 35s 356ms/step - loss: 0.0068
Epoch 9/50
97/97 [==============================] - 35s 359ms/step - loss: 0.0064
Epoch 10/50
97/97 [==============================] - 35s 359ms/step - loss: 0.0063
Epoch 11/50
97/97 [==============================] - 35s 362ms/step - loss: 0.0062
Epoch 12/50
97/97 [==============================] - 35s 362ms/step - loss: 0.0061
Epoch 13/50


In [25]:
# Model evaluation
#model_eval = cnn_model.evaluate(x_test, y_test)
i = 0 # Random value from test data set to check model accuracy
prediction=[]
for i in range(len(y_test)):
    image_arr = x_test[i]
    #print(image_arr.shape)
    #plt.imshow(image_arr)
    #plt.show()
    #image_fl = image_arr.astype("float32")
    #image_norm = image_fl/ 255.0
    #image_reshape = image_arr.reshape(2, image_arr.shape[0], image_arr.shape[1], image_arr.shape[2])
    image_pred = np.expand_dims(image_arr, axis=0)
    prediction.append(cnn_model.predict(image_pred))
    pred_angle = float(prediction[i])
    angle = y_test[i]
    dev = angle/pred_angle
    print(f'{i} Actual is {angle} Predicted is {pred_angle}')

#print(np.mean(prediction))

0 Actual is 0.0 Predicted is 0.000542917288839817
1 Actual is 0.0 Predicted is 0.017370518296957016
2 Actual is 0.0 Predicted is -0.0034478073939681053
3 Actual is 0.10000000149011612 Predicted is 0.045138128101825714
4 Actual is 0.0 Predicted is 0.07943874597549438
5 Actual is 0.0 Predicted is -0.02459646761417389
6 Actual is -0.0 Predicted is -0.026436101645231247
7 Actual is 0.0 Predicted is -0.01580851897597313
8 Actual is 0.0 Predicted is 0.0017873570322990417
9 Actual is 0.0 Predicted is -0.01000578235834837
10 Actual is 0.0 Predicted is 0.0926177054643631
11 Actual is 0.0 Predicted is -0.0015672780573368073
12 Actual is 0.10000000149011612 Predicted is 0.05133948102593422
13 Actual is 0.0 Predicted is 0.11641823500394821
14 Actual is 0.0 Predicted is 0.021058183163404465
15 Actual is 0.0 Predicted is -0.00165476743131876
16 Actual is 0.0 Predicted is 0.11414044350385666
17 Actual is 0.0 Predicted is 0.05883939936757088
18 Actual is 0.0 Predicted is 0.03085688129067421
19 Actual 

155 Actual is 0.0 Predicted is 0.01290092058479786
156 Actual is 0.0 Predicted is 0.09283173829317093
157 Actual is 0.0 Predicted is 0.0009877420961856842
158 Actual is 0.0 Predicted is 0.10575376451015472
159 Actual is 0.0 Predicted is -0.039171990007162094
160 Actual is 0.0 Predicted is -0.03694339841604233
161 Actual is -0.0 Predicted is -0.03464614227414131
162 Actual is 0.0 Predicted is 0.0017509488388895988
163 Actual is 0.10000000149011612 Predicted is 0.049878232181072235
164 Actual is 0.0 Predicted is 0.006982305087149143
165 Actual is 0.0 Predicted is -0.0066297343000769615
166 Actual is 0.0 Predicted is 0.006025201641023159
167 Actual is 0.10000000149011612 Predicted is 0.04012560471892357
168 Actual is 0.10000000149011612 Predicted is 0.048791274428367615
169 Actual is 0.10000000149011612 Predicted is 0.02233763597905636
170 Actual is 0.0 Predicted is 0.01026880368590355
171 Actual is -0.20000000298023224 Predicted is -0.03076765686273575
172 Actual is 0.0 Predicted is -0.0

302 Actual is -0.0 Predicted is -0.014611332677304745
303 Actual is 0.0 Predicted is 0.04340910539031029
304 Actual is 0.0 Predicted is -0.005273583345115185
305 Actual is 0.0 Predicted is -0.002295847050845623
306 Actual is 0.0 Predicted is 0.003981822170317173
307 Actual is 0.0 Predicted is 0.05032149702310562
308 Actual is 0.0 Predicted is 0.07286893576383591
309 Actual is 0.0 Predicted is -0.035832084715366364
310 Actual is 0.0 Predicted is -0.0569748692214489
311 Actual is 0.0 Predicted is 0.00900491513311863
312 Actual is 0.20000000298023224 Predicted is 0.1438910812139511
313 Actual is 0.0 Predicted is -0.049331579357385635
314 Actual is 0.0 Predicted is -0.011645757593214512
315 Actual is -0.0 Predicted is -0.013606696389615536
316 Actual is -0.10000000149011612 Predicted is -0.050740230828523636
317 Actual is 0.0 Predicted is -0.013507676310837269
318 Actual is -0.0 Predicted is -0.10805417597293854
319 Actual is 0.0 Predicted is -0.04210088774561882
320 Actual is 0.0 Predicte

In [24]:
# Save model to disk
model_name = 'bhv_clone_track05'
full_model_path = f'{base_model_path}/{model_name}.h5'
#print(model_name)
cnn_model.save(full_model_path)

In [ ]:
print(f'Model {full_model_path} saved')